# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that we've discussed some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modelling pipeline.  

With this pipeline, yoiu will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modelling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. We've discussed the bias variance tradeoff, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modelling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

## Loading the Data

In [27]:
#Your code here; load and preview the dataset
import pandas as pd
df = pd.read_csv('loan_final.csv')
df.head().T
df.dropna()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.00000,36 months,10.65%,162.87,B,10+ years,RENT,24000.00,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.00000,60 months,15.27%,59.83,C,< 1 year,RENT,30000.00,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.00000,36 months,15.96%,84.33,C,10+ years,RENT,12252.00,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.00000,36 months,13.49%,339.31,C,10+ years,RENT,49200.00,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.00000,60 months,12.69%,67.79,B,1 year,RENT,80000.00,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual
5,5000.0,5000.00000,36 months,7.90%,156.46,A,3 years,RENT,36000.00,Source Verified,Fully Paid,wedding,AZ,12.0,5632.210000,Individual
6,7000.0,7000.00000,60 months,15.96%,170.08,C,8 years,RENT,47004.00,Not Verified,Fully Paid,debt_consolidation,NC,11.0,10137.840010,Individual
7,3000.0,3000.00000,36 months,18.64%,109.43,E,9 years,RENT,48000.00,Source Verified,Fully Paid,car,CA,4.0,3939.135294,Individual
8,5600.0,5600.00000,60 months,21.28%,152.39,F,4 years,OWN,40000.00,Source Verified,Charged Off,small_business,CA,13.0,647.500000,Individual
9,5375.0,5350.00000,60 months,12.69%,121.45,B,< 1 year,RENT,15000.00,Verified,Charged Off,other,TX,3.0,1484.590000,Individual


## Defining the Problem

Set up the problem by defining X and Y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Be sure to use dummy variables for categorical variables and to normalize numerical quanitities. Be sure to also remove any rows with null data.  

For Y, we are looking to build a model to predict the total payment received for a loan.

In [28]:
# Your code here; appropriately define X and Y using dummy variables and normalization for preprocessing.
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

X = df[['loan_amnt', 'home_ownership', 'funded_amnt_inv', 'verification_status',
       'emp_length', 'installment', 'annual_inc']]
ohe = OneHotEncoder()
ss = StandardScaler()

enc_ho = X['home_ownership']
enc_vs = X['verification_status']
enc_emp = X['emp_length']
enc_ho = enc_ho.values.reshape(-1,1)
enc_vs = enc_vs.values.reshape(-1,1)
enc_emp = enc_emp.values.reshape(-1, 1)
ohe.fit(enc_ho)
enc_ho = pd.DataFrame(ohe.transform(enc_ho).toarray(), columns=ohe.get_feature_names(['home_ownership']))
ohe.fit(enc_vs)
enc_vs = pd.DataFrame(ohe.transform(enc_vs).toarray(), columns=ohe.get_feature_names(['verification_status']))
ohe.fit(enc_emp)
enc_emp = pd.DataFrame(ohe.transform(enc_emp).toarray(), columns=ohe.get_feature_names(['emp_length']))

X_s = X.drop(['home_ownership', 'verification_status', 'emp_length'],axis=1)
X_s = pd.DataFrame(ss.fit_transform(X_s), columns=X_s.columns)

X = pd.concat([X_s, enc_ho, enc_vs], axis=1)

y = df['total_pymnt']



ValueError: Input contains NaN

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modelling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [29]:
# Your code here; generate a hold out test set for final model evaluation. Use random seed 123.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=123)

## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. Be sure your function returns performance metrics regarding the training and validation sets.

In [32]:
#Your code here; define a function to evaluate a model object using K folds cross validation.
from sklearn.model_selection import KFold

def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    colors = sns.color_palette("Set2")
    kf = KFold(n_splits=10,shuffle=True)
    
    fig, axes = plt.subplots(2, 5, figsize=(12,8))
    validation_scores = []
    for i, (train_index, test_index) in enumerate(kf.split(features_train)):
        row = i//5
        column = i%5
        
        X_train, X_val = features_train.iloc[train_index], features_train.iloc[test_index]
        y_train, y_val = labels_train.iloc[train_index], labels_train.iloc[test_index]
        
        model = model_obj
        hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=32,
                        verbose=0)
        
        validation = model.evaluate(X_val, y_val)
        validation_scores.append(validation)
        ax = axes[row,col]
        k = 'val_loss'
        d = hist.history[k]
        ax.plot(d, label=k, colors=colors[1])
        ax.set_title(f'Fold {i+1} Validation')
    plt.subplots_adjust(left=None, right=None, bottom=None, top=None, wspace=.5, hspace=None)
    plt.legend(bbox_to_anchor=(1, 1))
    
    validation_score = np.mean(validation_scores)
    print(f'Mean Validation Score: {validation_score}')
    print(f'STD of Validation Scores: {np.std(validation_scores)}')
    return validation_score

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model.   

In general, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. We set up a k-folds cross-validation for this task since the dataset is not overly large.  

Build an initial sequential model with 2 hidden relu layers. The first should have 7 hidden units, and the second 10 hidden units. Finally, add a third layer with a linear activation function to output our predictions for the total loan payment. 

In [ ]:
#Your code here; define and compile an initial model as described
from keras import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense())

## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [ ]:
#Your code here; use your k-folds function to evaluate the baseline model.

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.  

For example, here are some timing notes on potential experiments run on a Macbook Pro 3.1 GHz Intel Core i5 with 16gb of RAM:

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 100 epochs took approximately 15 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 25 units per hidden layer and 100 epochs took approximately 25 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 250 epochs took approximately 45 minutes to train and validate


In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your expirements above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modelling process)!

In [ ]:
#Your code here; final model training on entire training set followed by evaluation on hold-out data

## Additional Resources

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

## Summary

In this lab, we investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. We began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, we implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, we'll begin to investigate other neural network architectures such as CNNs.